In [1]:
pip install --upgrade coiled dask bokeh 

Requirement already up-to-date: coiled in /Users/cody/miniconda3/envs/dkdc/lib/python3.8/site-packages (0.0.25)
     |████████████████████████████████| 848 kB 2.4 MB/s 
Requirement already up-to-date: bokeh in /Users/cody/miniconda3/envs/dkdc/lib/python3.8/site-packages (2.2.1)
  Attempting uninstall: dask
    Found existing installation: dask 2.27.0
    Uninstalling dask-2.27.0:
      Successfully uninstalled dask-2.27.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!coiled login --token <>

In [2]:
import coiled 

cluster = coiled.Cluster(
    n_workers=20,
)
cluster

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found


In [3]:
from dask.distributed import Client

c = Client(cluster)
c

Client Scheduler: tls://ec2-3-137-183-41.us-east-2.compute.amazonaws.com:8786 Dashboard: https://cloud.coiled.io/dashboard/1060/status,Cluster Workers: 20 Cores: 80 Memory: 343.60 GB


In [4]:
!pip install --upgrade git+https://github.com/dask/adlfs fsspec

  Cloning https://github.com/dask/adlfs to /private/var/folders/gk/kpsykczx509cvpggyml79rsc0000gn/T/pip-req-build-2ntqpmo9
Requirement already up-to-date: fsspec in /Users/cody/miniconda3/envs/dkdc/lib/python3.8/site-packages (0.8.3)
  Created wheel for adlfs: filename=adlfs-0.5.3+24.gad4074b-py3-none-any.whl size=18754 sha256=63b04afc87536e182676815a05d8a427420116631c42dbd120202a296ebd6b83
  Stored in directory: /private/var/folders/gk/kpsykczx509cvpggyml79rsc0000gn/T/pip-ephem-wheel-cache-3rs42805/wheels/a1/6c/2d/3b988aa0fd36594a0291addc195e10386681f4f354c95fbe75
Successfully built adlfs
  Attempting uninstall: adlfs
    Found existing installation: adlfs 0.5.3+18.g65de4a6
    Uninstalling adlfs-0.5.3+18.g65de4a6:
      Successfully uninstalled adlfs-0.5.3+18.g65de4a6


In [5]:
def update_packages():
    import os
    os.system('pip install --upgrade git+https://github.com/dask/adlfs fsspec')

In [6]:
import dask 

to_compute = [dask.delayed(update_packages) for i in range(20)]
c.compute(to_compute)

[<Future: pending, key: update_packages-11d25768-0457-4cf3-a12c-9a170df86a87>,
 <Future: pending, key: update_packages-d14a8cb8-35a0-4f8f-a548-dcaed173109b>,
 <Future: pending, key: update_packages-e11b7a2d-d99f-4b9a-b952-de3e4d0f531b>,
 <Future: pending, key: update_packages-2c842bed-beec-4d32-99f0-4641466992ad>,
 <Future: pending, key: update_packages-a996a493-839d-43d9-9a35-eb63c3436d04>,
 <Future: pending, key: update_packages-3dd89631-788c-414c-a1f0-868c390bcba6>,
 <Future: pending, key: update_packages-380d6727-e99c-49f7-9fa5-fb628765bfce>,
 <Future: pending, key: update_packages-19a46924-2fa4-4a10-acfb-f4aeb39e50df>,
 <Future: pending, key: update_packages-dd5526d7-1d2a-4d33-b2fa-60ab51ab7a17>,
 <Future: pending, key: update_packages-888a509e-6bd2-4145-a153-c82800346183>,
 <Future: pending, key: update_packages-c4bda028-af30-40e4-ac40-959466835e62>,
 <Future: pending, key: update_packages-24ff8774-e26a-4b40-b46a-619839248e5e>,
 <Future: pending, key: update_packages-52ea546d-b9a

In [7]:
from adlfs import AzureBlobFileSystem

storage_options = {'account_name': 'azureopendatastorage'}

fs = AzureBlobFileSystem(**storage_options)
fs.ls('isdweatherdatacontainer')

['isdweatherdatacontainer/ISDWeather/',
 'isdweatherdatacontainer/ISDWeatherAll',
 'isdweatherdatacontainer/ISDWeatherAll/',
 'isdweatherdatacontainer/ISDWeatherBackup',
 'isdweatherdatacontainer/ISDWeatherBackup/',
 'isdweatherdatacontainer/ISDWeatherBackup_20200115/',
 'isdweatherdatacontainer/ISDWeatherTest/',
 'isdweatherdatacontainer/fama',
 'isdweatherdatacontainer/fama/']

In [8]:
files = fs.glob('isdweatherdatacontainer/ISDWeather/year=*/month=*/*.parquet')
files[-5:]

['isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00003-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2443-9.c000.snappy.parquet',
 'isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00004-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2448-9.c000.snappy.parquet',
 'isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00005-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2445-9.c000.snappy.parquet',
 'isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00006-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2449-9.c000.snappy.parquet',
 'isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00007-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2444-9.c000.snappy.parquet']

In [15]:
files = [f'az://{file}' for file in files]
files[-5:]

['az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00003-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2443-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00004-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2448-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00005-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2445-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00006-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2449-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00007-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2444-9.c000.snappy.parquet']

In [16]:
import dask.dataframe as dd 

data_url = 'az://isdweatherdatacontainer/ISDWeather/year=*/month=*/*.parquet'

df = dd.read_parquet(
    files, 
    storage_options=storage_options, 
    columns=['temperature']
    )
df

,temperature
npartitions=1224,
,float64
,...
...,...
,...
,...


In [17]:
df.head()

KilledWorker: ("('read-parquet-head-1-5-read-parquet-a8268b7d7b80d2e1123680be59951852', 0)", <Worker 'tls://10.2.11.142:39213', name: lostmygithubaccount-1060-worker-15-ebfbaa, memory: 0, processing: 1>)

In [ ]:
df = df.persist()

In [18]:
c.restart()
c

Client Scheduler: tls://ec2-3-137-183-41.us-east-2.compute.amazonaws.com:8786 Dashboard: https://cloud.coiled.io/dashboard/1060/status,Cluster Workers: 20 Cores: 80 Memory: 343.60 GB


In [19]:
# this runs fine

import dask.dataframe as dd

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

df.groupby("passenger_count").tip_amount.mean().compute()

passenger_count
0    2.122789
1    2.206790
2    2.214306
3    2.137775
4    2.023804
5    2.235441
6    2.221105
7    6.675962
8    7.111625
9    7.377822
Name: tip_amount, dtype: float64